# FX AI Аналитика — исследовательский ноутбук

Этот ноутбук демонстрирует, как программно использовать функции из `app.py` без Streamlit UI: загрузить данные, обучить модели, посчитать новости и события, получить итоговый сигнал и визуализировать цену и ATR(14).

Ниже идут шаги: импорт, конфигурация инструмента, расчёты и графики, а в конце — текстовые рекомендации и возможные узкие места текущего подхода.

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import plotly.graph_objects as go

sys.path.append(os.path.abspath("."))

from app import (
    load_price_data,
    add_features,
    add_targets,
    train_models,
    compute_news_sentiment,
    fetch_future_economic_events,
    score_future_events,
    detect_patterns,
    build_price_chart,
    build_atr_chart,
    build_prediction_chart,
    build_classification_chart,
    build_classification_comparison_chart,
    combine_signals,
    get_atr_volatility_info,
    enrich_signals_with_atr,
    get_signals_for_ticker,
)

## 1. Конфигурация инструмента

Задаём тикер, человеко-понятное имя, глубину истории и таймфрейм. Профиль (консервативный/агрессивный и т.п.) можно оставить `None`, тогда будут использованы настройки по умолчанию из `INSTRUMENT_SETTINGS` внутри `app.py`.

In [ ]:
ticker = "EURUSD=X"
instrument_name = "EUR/USD"
years = 5
interval = "1d"
profile = None

config = {
    "ticker": ticker,
    "instrument_name": instrument_name,
    "years": years,
    "interval": interval,
    "profile": profile,
}
config

## 2. Быстрый вызов единого пайплайна

Используем высокоуровневую функцию `get_signals_for_ticker`, которая внутри:
- загружает ценовую историю через `yfinance`;
- строит фичи, таргеты и обучает модели;
- считает новости и будущие события;
- строит паттерны и интегральный скоринг;
- добавляет в сигнал последние значения ATR(14) и уровень волатильности.

Это удобная точка входа для API и мониторинга.

In [ ]:
signal_payload = get_signals_for_ticker(
    ticker=ticker,
    instrument_name=instrument_name,
    years=years,
    interval=interval,
    profile=profile,
)
signal_payload

## 3. Детальный шаг за шагом пайплайн

Теперь повторим ключевые шаги раздельно: загрузка данных, построение признаков, обучение моделей и объединение сигналов. Это полезно для отладки и экспериментов.

In [ ]:
df_raw = load_price_data(ticker, years=years, interval=interval)
df_full = add_features(df_raw)
atr_info = get_atr_volatility_info(df_full)

horizon = 7
lower_q = 0.33
upper_q = 0.66

df_model = add_targets(
    df_full.copy(),
    horizon=horizon,
    lower_q=lower_q,
    upper_q=upper_q,
)
model_data = train_models(df_model)
metrics = model_data.get("metrics")
df_full.tail()

## 4. Новости, события, паттерны и финальный сигнал

Считаем новостной фон, будущие макро-события, паттерны по цене и собираем финальный интегральный сигнал, включая информацию по ATR(14).

In [ ]:
news_items, news_score = compute_news_sentiment(ticker, instrument_name)
future_events = fetch_future_economic_events(days_ahead=7)
future_events_score = score_future_events(future_events)
patterns = detect_patterns(df_full["Close"])

model_weights = metrics.get("weights") if metrics else None
price_model = "hybrid" if metrics and "hybrid" in metrics else "lstm"
cls_conf_threshold = 0.55

signals = combine_signals(
    df_full,
    model_data,
    news_score,
    patterns,
    future_events_score,
    model_weights,
    price_model,
    classifier_override=None,
    cls_conf_threshold=cls_conf_threshold,
)
signals = enrich_signals_with_atr(signals, atr_info)
signals

## 5. Визуализация: цена, ATR(14), прогнозы и классификация

Используем те же функции построения графиков, что и внутри Streamlit-приложения, но показываем их напрямую в ноутбуке через Plotly.

In [ ]:
price_fig = build_price_chart(df_full, instrument_name, patterns=patterns)
atr_fig = build_atr_chart(df_full, instrument_name)
pred_fig = build_prediction_chart(df_model, model_data, price_model, model_weights)
class_fig = build_classification_chart(df_model, model_data, classifier_override=None)

price_fig.show()
atr_fig.show()
pred_fig.show()
class_fig.show()

In [ ]:
if metrics:
    comparison_fig = build_classification_comparison_chart(
        df_model,
        model_data,
        model_weights,
        cls_conf_threshold,
    )
    comparison_fig.show()

## 6. Рекомендации и возможные ошибки/ограничения подхода

Ниже несколько практических замечаний по текущему проекту:

1. **Данные и зависимость от Yahoo Finance**  
   Вся ценовая история берётся через `yfinance`. При изменении API или временной недоступности сервиса модели не смогут обновляться. Имеет смысл предусмотреть кэширование данных (например, в локальных CSV или базе) и fallback-набор данных для критичных инструментов.

2. **Очень простой сентимент-анализ новостей**  
   Текущий подход к новостям опирается на простой поиск ключевых слов и не учитывает контекст, сарказм, многозначность слов. Для повышения качества сигналов можно постепенно перейти на более мощную модель (например, готовый sentiment-классификатор или небольшую языковую модель, применяемую к заголовкам/аннотациям).

3. **Эвристический детект паттернов**  
   Функция `detect_patterns` базируется на наборе эвристик и z-оценок; она может давать как пропуски важных фигур, так и ложные срабатывания. При масштабировании проекта полезно будет:
   - отделить паттерны, проверенные статистически, от гипотез;
   - залогировать частоту появления каждого паттерна и последующую доходность.

4. **ATR(14) и интерпретация волатильности**  
   Добавление ATR(14) и текстовых уровней волатильности (низкая/средняя/высокая) улучшает понимание риска, но пороги сейчас задаются автоматически по квантилям исторического ATR. Для более строгого risk-management стоит:
   - явно задавать пороги ATR для каждого инструмента (на основе исторического анализа);
   - связать уровень волатильности с размером позиции или фильтрами сигналов (например, игнорировать слабые BUY при экстремально высокой волатильности).

5. **Смешение нескольких моделей и риск переобучения**  
   Гибрид LGBM+LSTM и дополнительный SVC дают гибкий ансамбль, но усложняют интерпретацию. Важно периодически пересматривать качество каждой модели на перекрёстной проверке и логировать метрики по времени, чтобы отслеживать деградацию. Также стоит добавить проверку устойчивости к сдвигам распределения (например, изменение волатильности рынка).

6. **Бизнес-логика порогов для рекомендаций BUY/SELL/HOLD**  
   Порог `score > 0.4` для BUY и `< -0.4` для SELL выбран эвристически. Имеет смысл:
   - оптимизировать пороги на историческом тесте (максимизация `Sharpe`, `Sortino` или другой метрики);
   - держать пороги зависящими от профиля риска пользователя (консервативный/агрессивный).

7. **Инфраструктура и деплой**  
   Так как приложение завязано на Streamlit и FastAPI, важно чётко разделять окружения: одно для UI, другое для API. В ноутбуке лучше не полагаться на наличие Streamlit-сервера, а использовать только функции из `app.py`, как сделано выше. Это повышает переносимость и облегчает интеграцию с внешними пайплайнами (Airflow, Prefect и т.п.).

Этот ноутбук можно использовать как отправную точку для быстрых экспериментов и отладки, постепенно дополняя его более продвинутыми экспериментами (новые фичи, другие горизонты, альтернативные модели).